<a href="https://colab.research.google.com/github/1kaiser/Snow-cover-area-estimation/blob/main/tiff2array.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path
import re

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

def get_gain_band(input_file):
    """get GAIN_BAND from meta file (*.tif.txt)"""
     # define file name of *.tif.txt
    ifile_txt = re.sub(r'.tif', '.tif.txt', input_file)
    ld = open(ifile_txt)
    lines = ld.readlines()
    ld.close()
    
    gain_band = []
    for line in lines:
        if line.find("GAIN_BAND") >= 0:
             gain_band.append(float((re.split(' ', line)[1]).strip()))
    return gain_band

def tif2array(input_file, calc_gain=True):
    """
    read GeoTiff and convert to numpy.ndarray.
    Inputs:
        input_file (str) : the name of input GeoTiff file.
        calc_gain (bool) : wheter calc GAIN to DN  or not (defaul:True).
    return:
        image(np.array) : image for each bands
        dataset : for gdal's data drive.
    """
    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)
    # Allocate our array using the first band's datatype
    image_datatype = dataset.GetRasterBand(1).DataType
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                     dtype=float)
    
    if calc_gain == True:
        # get gain
        gain = get_gain_band(input_file)
    
    # Loop over all bands in dataset
    for b in range(dataset.RasterCount):
        # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
        band = dataset.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        if calc_gain == True:
            # calc gain value for each bands
            image[:, :, b] = band.ReadAsArray() * gain[b]
        else:
            image[:, :, b] = band.ReadAsArray()
    return image, dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
!cp -r /content/gdrive/MyDrive/OUT/data/data20230320 /content/

In [1]:
temp_dir = r'/content/'
images = []
path_list = ['C1/download.B11.tif',
             'C1/download.B12.tif',
             'C1/download.B2.tif',
             'C1/download.B3.tif',
             'C1/download.B4.tif',
             'C1/download.B5.tif',
             'C1/download.B6.tif',
             'C1/download.B7.tif',
             'C1/download.B8A.tif',
             'C1/download.B8x.tif',]

In [ ]:
temp_dir = r'/content/'
images = []
path_list = ['data20230320/download.B11.tif',
             'data20230320/download.B12.tif',
             'data20230320/download.B2.tif',
             'data20230320/download.B3.tif',
             'data20230320/download.B4.tif',
             'data20230320/download.B5.tif',
             'data20230320/download.B6.tif',
             'data20230320/download.B7.tif',
             'data20230320/download.B8A.tif',
             'data20230320/download.B8x.tif',]

In [4]:
from osgeo import gdal


lon , lat = [], []  
v1 =[]

for i in path_list:
  pathout = temp_dir+str(i)
  print(pathout)
  if(path_list.index(i)==0):
    v1 = tif2array(pathout, 0)[0].reshape(-1,1)
    # Open tif file
    ds = gdal.Open(pathout)
    # GDAL affine transform parameters, According to gdal documentation xoff/yoff are image left corner, a/e are pixel wight/height and b/d is rotation and is zero if image is north up. 
    xoff, a, b, yoff, d, e = ds.GetGeoTransform()
    print(xoff, a, b, yoff, d, e)
    def pixel2coord(x, y):
      # ""Returns global coordinates from pixel x, y coords""
      xp = a * x + b * y + xoff
      yp = d * x + e * y + yoff
      return xp, yp

    # get columns and rows of your image from gdalinfo
    rows = ds.RasterYSize 
    colms = ds.RasterXSize 
    for row in  range(0,rows):
      for col in  range(0,colms): 
        lat.append(pixel2coord(col,row)[0])
        lon.append(pixel2coord(col,row)[1])
  else:
    v1 = np.append(v1, tif2array(pathout, 0)[0].reshape(-1,1), axis = 1)
# agging latitude "lat" & longitde "lon" column at the end
v1 = np.column_stack((v1, np.column_stack((lat, lon))))


/content/C1/download.B11.tif
76.972043626781 8.983152841195255e-05 0.0 32.40229910532168 0.0 -8.983152841195288e-05
/content/C1/download.B12.tif
/content/C1/download.B2.tif
/content/C1/download.B3.tif
/content/C1/download.B4.tif
/content/C1/download.B5.tif
/content/C1/download.B6.tif
/content/C1/download.B7.tif
/content/C1/download.B8A.tif
/content/C1/download.B8x.tif


In [5]:
import pandas as pd
arr = v1
 
# display the array
# print(arr)
 
# convert array into dataframe

d = {'b11','b12','b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8a', 'b8', 'lat', 'lon'}

DF = pd.DataFrame(arr)
# DF.columns = d
DF
# print(DF)
# # save the dataframe as a csv file
# DF.to_csv("data1.csv")

,0,1,2,3,4,5,6,7,8,9,10,11
0,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,76.972044,32.402299
1,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,76.972133,32.402299
2,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,76.972223,32.402299
3,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,76.972313,32.402299
4,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,76.972403,32.402299
...,...,...,...,...,...,...,...,...,...,...,...,...
19180483,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,77.449678,32.078636
19180484,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,77.449768,32.078636
19180485,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,77.449858,32.078636
19180486,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,77.449947,32.078636


In [6]:
#Copy and execute the following code in a new Google Colab notebook launch to run the model.

!python -m pip install tensorflow tensorflow_decision_forests -U -qq

# Transfer the model from Google Drive to Colab
from google.colab import drive
drive.mount("/content/gdrive")
!cp "/content/gdrive/My Drive/simple_ml_for_sheets/Ice height from bands" ydf_model
  
# Prepare and load the model with TensorFlow
import tensorflow as tf
import tensorflow_decision_forests as tfdf


tfdf.keras.yggdrasil_model_to_keras_model("ydf_model", "tfdf_model")
model = tf.keras.models.load_model("tfdf_model")

# # Make predictions with the model
# examples = {
#   "b11" : DF[0],
#   "b12" : DF[1],
#   "b2" : DF[2],
#   "b3" : DF[3],
#   "b4" : DF[4],
#   "b5" : DF[5],
#   "b6" : DF[6],
#   "b7" : DF[7],
#   "b8a" : DF[8],
#   "b8" : DF[9],
#   "lat" : DF[10],
#   "lon" : DF[11],
# }
# a = model.predict_step(examples)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [7]:
height =[]
for i in range(0, len(DF[0])):
  examples = {
  "b11" : [np.array(DF[0][i])],
  "b12" : [np.array(DF[1][i])],
  "b2" : [np.array(DF[2][i])],
  "b3" : [np.array(DF[3][i])],
  "b4" : [np.array(DF[4][i])],
  "b5" : [np.array(DF[5][i])],
  "b6" : [np.array(DF[6][i])],
  "b7" : [np.array(DF[7][i])],
  "b8a" : [np.array(DF[8][i])],
  "b8" : [np.array(DF[9][i])],
  "lat" : [np.array(DF[10][i])],
  "lon" : [np.array(DF[11][i])],
  }
  a = np.array(model.predict_step(examples))
  height.append(a[0][0])
print(np.asarray(height))
print(height)
height = np.column_stack((np.column_stack((lat, lon)), height))
# saving the predicted heght data to panda dataframe
DHeight = pd.DataFrame(height)
# DF.columns = d
DHeight
# print(DF)
# # save the dataframe as a csv file
# DF.to_csv("data1.csv")

KeyboardInterrupt: ignored

In [8]:
height =[]
for i in range(0, len(DF[0])):
  examples = {
  "b11" : [np.array(DF[0][i])],
  "b12" : [np.array(DF[1][i])],
  "b2" : [np.array(DF[2][i])],
  "b3" : [np.array(DF[3][i])],
  "b4" : [np.array(DF[4][i])],
  "b5" : [np.array(DF[5][i])],
  "b6" : [np.array(DF[6][i])],
  "b7" : [np.array(DF[7][i])],
  "b8a" : [np.array(DF[8][i])],
  "b8" : [np.array(DF[9][i])],
  "lat" : [np.array(DF[10][i])],
  "lon" : [np.array(DF[11][i])],
  }
  a = np.array(model.predict_step(examples))
  height.append(a[0][0])
print(np.asarray(height))
print(height)

height = np.column_stack((np.column_stack((lat, lon)), height))
DHeight = pd.DataFrame(height)
# DF.columns = d
DHeight
# print(DHeight)
# # save the dataframe as a csv file
# DHeight.to_csv("data1.csv")

[2800.0225 2800.0225 2800.0225 2800.0225 2800.0225 2800.0225 2800.0225
 2800.0225 2800.0225 2800.0225]
[2800.0225, 2800.0225, 2800.0225, 2800.0225, 2800.0225, 2800.0225, 2800.0225, 2800.0225, 2800.0225, 2800.0225]


ValueError: ignored

In [12]:
np.array(DF[0])[1]


-3.4028230607370965e+38

In [6]:

examples = tfdf.keras.pd_dataframe_to_tf_dataset(DF)
a = model.predict_step(examples)

ValueError: ignored

In [13]:
print(np.array(a))

[[3565.8792]
 [3565.8792]
 [3565.8792]]
